In [1]:
# IMPORTS
from manufacturing_company.src.common.const import *
from manufacturing_company.src.features.employee_activity import *
from manufacturing_company.src.features.neighborhood_variability import *
import pandas as pd

In [24]:
# CALCULATE OVERTIME AND WORK AT WEEKEND
for i in range(1, SIZE + 1):
    df = pd.read_csv('manufacturing_company/data/intermediate/03_minimum_activity/' + str(i) + '_months_communication.csv', sep=';', parse_dates=[EVENT_DATE])
    df_overtime = calculate_overtime(df, from_hour=17, to_hour=5)
    df_weekend = calculate_work_at_weekend(df)
    df_merged = pd.merge(df_overtime, df_weekend, on=SENDER)
    
    df_network = pd.read_csv('manufacturing_company/data/intermediate/04_networks/' + str(i) + '_months_network.csv', sep=';')
    df_merged = pd.merge(df_network, df_merged, left_on=ID, right_on=SENDER)
    
    sender, recipient, sender_recipient = calculate_neighborhood_variability(df, df_merged[ID])
    
    sender = pd.DataFrame.from_dict(sender, orient='index', columns=[NEIGHBORHOOD_VARIABILITY_SENDER])
    sender.index.name = ID
    recipient = pd.DataFrame.from_dict(recipient, orient='index', columns=[NEIGHBORHOOD_VARIABILITY_RECIPIENT])
    recipient.index.name = ID
    sender_recipient = pd.DataFrame.from_dict(sender_recipient, orient='index', columns=[NEIGHBORHOOD_VARIABILITY_ALL])
    sender_recipient.index.name = ID
    
    df_merged = pd.merge(df_merged, sender, on=ID)
    df_merged = pd.merge(df_merged, recipient, on=ID)
    df_merged = pd.merge(df_merged, sender_recipient, on=ID)
    
    df_merged.to_csv('manufacturing_company/data/intermediate/'
                     '05_features/' + str(i) + '_months_features.csv', sep=';', index=False)
